# How to use `unfold`

Author: [Romain Sacchi](romain.sacchi@psi.ch)

Github repo: https://github.com/romainsacchi/

## fold
Several `brightway` databases are "folded" in a data package.

In [1]:
from unfold import Fold
import bw2data
bw2data.projects.set_current("democarmaup") # <-- name of the brightway project containing both the source database and the databases to fold

In [2]:
f = Fold()

In [3]:
f.fold()

Give a name for this datapackage:  some datapackage
Give a short description for this datapackage:  this is a test


The following databases are available in your project:
+-----+--------------------------------+
| No. |            Database            |
+-----+--------------------------------+
|  1  |           biosphere3           |
|  2  |            cutoff38            |
|  3  |           cutoff371            |
|  4  |        buses_bw_ei_371         |
|  5  |    passenger_cars_bw_ei_371    |
|  6  |        trucks_bw_ei_371        |
|  7  |             cobalt             |
|  8  |            cutoff36            |
|  9  |         carculator db          |
|  10 |    image - SSP2-Base - 2025    |
|  11 |           steel 2025           |
|  12 |    image - SSP2-Base - 2050    |
|  13 | ecoinvent_image_SSP2-Base_2025 |
|  14 | ecoinvent_image_SSP2-Base_2050 |
+-----+--------------------------------+



Indicate the no. of the reference database:  2
Indicate the system model of cutoff38:  cutoff
Indicate the version of cutoff38:  3.8
Indicate the no. of the databases you would like to fold, separated by a comma:  13, 14
Give a short description for ecoinvent_image_SSP2-Base_2025:  some scenario for 2025
Give a short description for ecoinvent_image_SSP2-Base_2050:  another one for 2050


Getting activity data


100%|██████████████████████████████████████████████████████████████████████████████████████████| 19569/19569 [00:00<00:00, 312285.59it/s]


...

Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|███████████████████████████████████████████████████████████████████████████████████████████| 4451/4451 [00:00<00:00, 5117556.77it/s]


Building scenario factor file...
Data package saved at some datapackage.zip


In [3]:
# alternatively, you can answer these questions directly by passing the following arguments
f.fold(
    package_name="a package name",
    package_description="some description",
    source="cutoff38",
    system_model="cut off",
    version="3.8",
    databases_to_fold=["ecoinvent_image_SSP2-Base_2025", "ecoinvent_image_SSP2-Base_2050"],
    descriptions=["some db", "some other db"],
)

Getting activity data


100%|█████████████████████████████████| 19569/19569 [00:00<00:00, 291383.41it/s]


Adding exchange data to activities


100%|████████████████████████████████| 630210/630210 [00:20<00:00, 30247.22it/s]


Filling out exchange data


...

Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████████████████████████████| 4451/4451 [00:00<00:00, 3405481.05it/s]


Building scenario factor file...
Data package saved at a package name.zip


## unfold
Here, we read a data package, and we unfold and write the folded databases back into a `brightway` project.
It will write one database per scenario to your `brightway` project.

In [1]:
from unfold import Unfold
import bw2data
bw2data.projects.set_current("democarmaup")

In [2]:
u = Unfold("a package name.zip")

The data package contains the following scenarios:
+-----+--------------------------------+---------------+
| No. |            Scenario            |  Description  |
+-----+--------------------------------+---------------+
|  0  | ecoinvent_image_SSP2-Base_2025 |    some db    |
|  1  | ecoinvent_image_SSP2-Base_2050 | some other db |
+-----+--------------------------------+---------------+

To unfold all scenarios, `unfold()`.
To unfold a specific scenario, `unfold(scenarios=[1,])`.


In [ ]:
u.unfold()

In [3]:
# alternatively, you can answer these questions directly by passing the following arguments
u.unfold(
    scenarios=[0],
    dependencies={
        "cutoff38": "cutoff38",
        "biosphere3": "biosphere3"
    }
)

Getting activity data


100%|█████████████████████████████████| 19569/19569 [00:00<00:00, 306043.19it/s]


Adding exchange data to activities


100%|████████████████████████████████| 630210/630210 [00:18<00:00, 33304.59it/s]


Filling out exchange data


...

Filling out exchange data


100%|██████████████████████████████████| 4451/4451 [00:00<00:00, 5162844.88it/s]


Creating database for scenario ecoinvent_image_SSP2-Base_2025...
Writing database for scenario ecoinvent_image_SSP2-Base_2025...
Database ecoinvent_image_SSP2-Base_2025 already exists: it will be overwritten.
Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:21


Title: Writing activities to SQLite3 database:
  Started: 11/19/2022 16:17:25
  Finished: 11/19/2022 16:17:47
  Total time elapsed: 00:00:21
  CPU %: 97.30
  Memory %: 34.70
Created database: ecoinvent_image_SSP2-Base_2025


# unfold
## superstructure databases
`unfold` can also unfold [superstructure](https://link.springer.com/article/10.1007/s11367-021-01974-2) databases, to be used with [Activity Browser](https://github.com/LCA-ActivityBrowser/activity-browser).
It will write only one database, regardless of the number of scenarios.
It will also export a scenario difference spreadsheet to be read by Activity Browser.

In [3]:
u.unfold(
    scenarios=[0, 1],
    dependencies={
        "cutoff38": "cutoff38",
        "biosphere3": "biosphere3"
    },
    superstructure=True # <-- NEW
)

Getting activity data


100%|█████████████████████████████████| 19569/19569 [00:00<00:00, 296096.82it/s]


Adding exchange data to activities


100%|████████████████████████████████| 630210/630210 [00:20<00:00, 30374.03it/s]


Filling out exchange data


...

Filling out exchange data


100%|██████████████████████████████████| 4451/4451 [00:00<00:00, 5158565.10it/s]


Writing scenario difference file...
Scenario difference file exported to a package name.xlsx!
Writing superstructure database...


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:27


Title: Writing activities to SQLite3 database:
  Started: 11/19/2022 16:23:59
  Finished: 11/19/2022 16:24:27
  Total time elapsed: 00:00:27
  CPU %: 85.50
  Memory %: 31.25
Created database: a package name


## premise can also export datapackages

In [1]:
from premise import *
import bw2io, bw2data
import pandas as pd

In [2]:
bw2data.projects.set_current("democarmaup")

In [3]:
ndb = NewDatabase(
    scenarios = [
        {"model":"image", "pathway":"SSP2-Base", "year":2025},
        {"model":"image", "pathway":"SSP2-Base", "year":2050},
    ],
    source_db="cutoff38", # <- give the exact name of the database as registered in the project
    source_version="3.8",
    key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
)

premise v.(1, 3, 9)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [4]:
ndb.update_steel()
#ndb.update_electricity()


////////////////////////////// STEEL //////////////////////////////
Log of deleted steel datasets saved in /Users/romain/GitHub/premise/premise/data/logs
Log of created steel datasets saved in /Users/romain/GitHub/premise/premise/data/logs
Create steel markets for different regions
Create new steel production datasets and empty old datasets
Create pig iron production datasets
Done!
Log of deleted steel datasets saved in /Users/romain/GitHub/premise/premise/data/logs
Log of created steel datasets saved in /Users/romain/GitHub/premise/premise/data/logs
Create steel markets for different regions
Create new steel production datasets and empty old datasets
Create pig iron production datasets
Done!


In [5]:
ndb.write_superstructure_db_to_brightway()

Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Prepare database 2.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Building superstructure database...
Dropped 0 duplicate(s).
Scenario difference file exported to /Users/romain/GitHub/premise/premise/data/export/scenario diff files!
Done!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


Title: Writing activities to SQLite3 database:
  Started: 11/19/2022 16:05:48
  Finished: 11/19/2022 16:06:12
  Total time elapsed: 00:00:23
  CPU %: 88.20
  Memory %: 11.00
Created database: super_db_2022-11-19


In [5]:
ndb.write_datapackage()

Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Building scenario factor file...
Data package saved at /Users/romain/GitHub/premise/premise/data/export/datapackage/datapackage_2022-11-17.zip
